# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'requests'

## Generate Cities List

In [7]:
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=20)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=20)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

NameError: name 'citipy' is not defined

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# base_url in farenheit
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&"

cloudiness = []
country = []
date = []
max_temp = []
humidity = []
lat = []
lng = []
wind_speed = []
city_name = []

# Loop through cities making the API call
i = 1
for city in cities:
    query_url = base_url + "appid=" + weather_api_key + "&q=" + city
    response = requests.get(query_url).json()
    
    try:
        cloudiness.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        max_temp.append(response["main"]['temp_max'])
        humidity.append(response["main"]['humidity'])
        lat.append(response["coord"]["lat"])  
        lng.append(response["coord"]["lon"])
        wind_speed.append(response["wind"]["speed"])
        city_name.append(response["name"])
        print(f"Processing record {i}|{city}")
        i+=1
        
    except (KeyError):
        print("City name not available.")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# build the city data into a csv

city_df = pd.DataFrame({"City": city_name, 
                        "Cloudiness": cloudiness, 
                        "Country": country, 
                        "Date": date, 
                        "Humidity":humidity,
                        "Lat": lat,
                        "Lng": lng,
                        "Max Temp": max_temp,
                        "Wind Speed": wind_speed
                                         })

city_df = city_df.rename_axis('City ID')

city_df.head()

In [ ]:
city_df.to_csv("output_data/city_list.csv")

city_df.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humid_city = city_df.loc[city_df["Humidity"] > 100]

humid_city.head()
humid_city.count()

In [ ]:
#  Get the indices of cities that have humidity over 100%.
index_list = city_df[(city_df["Humidity"] > 100)].index
                                  
index_list

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_df = city_df.drop(index_list, inplace=False)
clean_city_df.count()

In [ ]:
lats = clean_city_df["Lat"]
max_temps = clean_city_df["Max Temp"]
humidity = clean_city_df["Humidity"]
cloudiness = clean_city_df["Cloudiness"]
wind_speed = clean_city_df["Wind Speed"]

# Export the City_Data into a csv
clean_city_df.to_csv(output_data_file, index_label="City_ID")


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [2]:
import datetime
city_date = datetime.datetime.fromtimestamp(int(city_df["Date"][0])).strftime('%Y-%m-%d')
city_date

NameError: name 'city_df' is not defined

In [3]:
plt.scatter(city_df["Lat"], city_df["Max Temp"], marker = "o", color = "navy", alpha = .75)
plt.grid()
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.title(f"City Latitude vs. Max Temperature ({city_date})")
plt.savefig("output_data/Latitude vs. Max Temp plot.png")
plt.show()

NameError: name 'city_df' is not defined

## Latitude vs. Humidity Plot

In [4]:
plt.scatter(city_df["Lat"], city_df["Humidity"], marker = "o", color = "navy", alpha = .75)
plt.grid()
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.title(f"City Latitude vs. Humidity ({city_date})")
plt.savefig("output_data/Latitude vs. Humidity plot.png")
plt.show()

NameError: name 'city_df' is not defined

## Latitude vs. Cloudiness Plot

In [5]:
plt.scatter(city_df["Lat"], city_df["Cloudiness"], marker = "o", color = "navy", alpha = .75)
plt.grid()
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.title(f"City Latitude vs. Cloudiness( {city_date})")
plt.savefig("output_data/Latitude vs. Cloudiness plot.png")
plt.show()

NameError: name 'city_df' is not defined

## Latitude vs. Wind Speed Plot

In [6]:
plt.scatter(city_df["Lat"], city_df["Wind Speed"], marker = "o", color = "navy", alpha = .75)
plt.grid()
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.title(f"City Latitude vs. Wind Speed ({city_date})")
plt.savefig("output_data/Latitude vs. Wind Speed plot.png")
plt.show()

NameError: name 'city_df' is not defined

## Linear Regression

In [9]:
# Linear regression function
def linear_regression(x_values,y_values):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values, y_values)
    plt.plot(x_values,regress_values,"r-")
    return [line_eq, rvalue]

# Line annotation function
def annotate(line_eq, a, b):
    plt.annotate(line_eq,(a,b),fontsize=15,color="red")

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [10]:
north = city_df.loc[city_df["Lat"] >= 0]
north_df = pd.DataFrame(north)

south = city_df.loc[city_df["Lat"] < 0]
south_df = pd.DataFrame(south)

NameError: name 'city_df' is not defined

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [8]:
equation = linear_regression(north_df["Lat"], north_df["Max Temp"])

annotate(equation[0], 10, 50)

plt.title(f"Northern Hemisphere - Max Temp vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Northern Hemisphere - Max Temp vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [8]:
equation = linear_regression(north_df["Lat"], north_df["Humidity"])
annotate(equation[0], 45, 20)

plt.title(f"Northern Hemisphere - Humidity vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Northern Hemisphere - Humidity vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [9]:
equation = linear_regression(south_df["Lat"], south_df["Humidity"])
annotate(equation[0], -50, 20)

plt.title(f"Southern Hemisphere - Humidity vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Southern Hemisphere - Humidity vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [10]:
equation = linear_regression(north_df["Lat"], north_df["Cloudiness"])
annotate(equation[0], 25, 60)

plt.title(f"Northern Hemisphere - Cloudiness vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Northern Hemisphere - Cloudiness vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [11]:
equation = linear_regression(south_df["Lat"], south_df["Cloudiness"])
annotate(equation[0], -50, 30)

plt.title(f"Southern Hemisphere - Cloudiness vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Southern Hemisphere - Cloudiness vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [12]:
equation = linear_regression(north_df["Lat"], north_df["Wind Speed"])
annotate(equation[0], 10, 25)

plt.title(f"Northern Hemisphere - Wind Speed vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Northern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [13]:
equation = linear_regression(south_df["Lat"], south_df["Wind Speed"])
annotate(equation[0], -30, 25)

plt.title(f"Southern Hemisphere - Wind Speed vs. Latitude Linear Regression ({city_date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"The r-squared is : {equation[1]**2}")

plt.savefig("output_data/Southern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")

NameError: name 'linear_regression' is not defined